The dataset contain 65536 points in a 3d point cloud file.
Voxel downsampling is done to reduce the number of points in the point cloud while preserving its overall structure. 
It works by grouping nearby points into voxels (small 3D cubes) and replacing them with a single representative point per voxel.
The file is saved to the disk with name smoothed.ply 

In [17]:
import open3d as o3d
import numpy as np

data=np.load("3d_shape_points_data.npz")
points = data['points']

print(f'shape of the numpy array {points.shape}')
print(points)

# Create an Open3D point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Apply MLS smoothing
pcd_smoothed = pcd.voxel_down_sample(voxel_size=0.5)

# o3d.visualization.draw_geometries([pcd_smoothed])


shape of the numpy array (65536, 3)
[[ 9.998086    0.          0.        ]
 [ 9.994803    0.24535865  0.        ]
 [ 9.985485    0.4905554   0.        ]
 ...
 [19.84671    -1.4639814  40.        ]
 [19.879618   -0.97662306 40.        ]
 [19.900478   -0.48852932 40.        ]]


visualizing the figure using slices of points from the point cloud file , the shape resembles to a cylinder with about 3 deformations on its surface
the deformation happens to be of the type misaligned points in the point cloud.
so outliers are removed from the file 

In [18]:
# remove outliers if any

cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=1)
filtered_pcd = pcd.select_by_index(ind)
# o3d.visualization.draw_geometries([filtered_pcd], window_name="Filtered Point Cloud")
pcd=filtered_pcd

In the next step I tried to create a mesh from the point clouds to apply some of the mesh processing on it .
There are 3 method more commonly used to solve this
Alpha shapes 

Ball pivoting 

Poisson surface reconstruction 

I have tried each of these 3 method but a better result was show with Ball pivoting 

The 3 deformations are clearly visible in the below mesh

In [19]:
# creating a mesh from the point cloud using ball pivoting method

radii = [0.1,0.2,0.3] 
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid( radius=0.1, max_nn=20)) # normals is required for ball pivoting method

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)

# Compute vertex normals
mesh.compute_vertex_normals()
o3d.io.write_triangle_mesh("deformed_mesh.ply", mesh)
o3d.visualization.draw_geometries([mesh])

In the next step I applied some smoothing operation on the mesh trying to resolve the deformations. 

In [20]:
# some preprocessing on the mesh
mesh = mesh.filter_smooth_laplacian(number_of_iterations=100)

# Recompute normals for better visualization
mesh.compute_vertex_normals()

mesh = mesh.filter_smooth_simple(number_of_iterations=5)
mesh = mesh.filter_smooth_taubin(number_of_iterations=5)
# Visualize the smoothed mesh
# o3d.visualization.draw_geometries([mesh])

o3d.io.write_triangle_mesh("temp_mesh.ply", mesh)

True

I tried to fill the holes created by removing the deformation using pymeshlab.
It was not effective as I imagined. It couldn't alter the shape much

In [21]:
# # to fill the holes in the mesh
import os
import pymeshlab

ms = pymeshlab.MeshSet()
ms.load_new_mesh("temp_mesh.ply")
ms.generate_alpha_wrap(alpha_fraction= 0.03, offset_fraction  = 0.001)
ms.save_current_mesh("dest_mesh.ply")

# Load the STL file
mesh = o3d.io.read_triangle_mesh("dest_mesh.ply")
# Visualize the mesh
o3d.visualization.draw_geometries([mesh])
os.remove("temp_mesh.ply")
os.remove("dest_mesh.ply")


Then I tried to implement the convex hull on the mesh which performed well. It could fill the holes  without altering the original shape much.


In [22]:
filled_mesh, _ = mesh.compute_convex_hull()

# Check if the new mesh is watertight
is_filled_watertight = filled_mesh.is_watertight
o3d.visualization.draw_geometries([filled_mesh])

Lastly I converted the mesh back to the point cloud to visualize the original shape i e, the cylinder.

In [23]:
# creating the point cloud back from the meshes
point_cloud = filled_mesh.sample_points_uniformly(number_of_points=10000)
o3d.io.write_point_cloud("original_shape.ply", point_cloud)
o3d.visualization.draw_geometries([point_cloud])